Zero shot results

In [ ]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Load your dataset
data = pd.read_csv('useeio.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Define a function to perform zero-shot classification and evaluate metrics
def zero_shot_classification_and_evaluation(texts, true_labels, candidate_labels):
    predictions = [classifier(text, candidate_labels)['labels'][0] for text in texts]
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')
    return accuracy, precision, recall, f1

# Define the classification tasks
tasks = {
    "Commodity or Industry Classification": {
        "texts": data['commodity_industry_description'].tolist(),
        "true_labels": data['commodity_industry_label'].tolist(),
        "candidate_labels": ["commodity", "industry"]
    },
    "Emission Type Identification": {
        "texts": data['emission_description'].tolist(),
        "true_labels": data['emission_type_label'].tolist(),
        "candidate_labels": ["CO2", "CH4", "N2O"]
    },
    "Impact Assessment": {
        "texts": data['impact_description'].tolist(),
        "true_labels": data['impact_label'].tolist(),
        "candidate_labels": ["low impact", "medium impact", "high impact"]
    },
    "Supply Chain Stage Identification": {
        "texts": data['supply_chain_stage_description'].tolist(),
        "true_labels": data['supply_chain_stage_label'].tolist(),
        "candidate_labels": ["production", "transportation", "distribution"]
    },
    "Geographical Region Identification": {
        "texts": data['geographical_region_description'].tolist(),
        "true_labels": data['geographical_region_label'].tolist(),
        "candidate_labels": ["North America", "Europe", "Asia"]
    },
    "Temporal Trend Prediction": {
        "texts": data['temporal_trend_description'].tolist(),
        "true_labels": data['temporal_trend_label'].tolist(),
        "candidate_labels": ["increase", "decrease", "stable"]
    }
}

# Perform zero-shot classification and collect results
results = {}
for task_name, task_data in tasks.items():
    accuracy, precision, recall, f1 = zero_shot_classification_and_evaluation(
        task_data['texts'], task_data['true_labels'], task_data['candidate_labels'])
    results[task_name] = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Display the results in a table
results_df = pd.DataFrame(results).T
print(results_df)

# Plot the results in a graph
metrics = ["accuracy", "precision", "recall", "f1"]
for metric in metrics:
    plt.plot(results_df.index, results_df[metric], marker='o', label=metric)

plt.xlabel("Tasks")
plt.ylabel("Scores")
plt.title("Comparison of Zero-Shot Classification Tasks")
plt.legend()
plt.xticks(rotation=45)
plt.show()


Results for supervised learning using classical models

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load your dataset
data = pd.read_csv('useeio.csv')

# Preprocess the data (example, adjust according to your dataset)
# Encoding categorical variables
label_encoders = {}
for column in ['commodity_industry_label', 'emission_type_label', 'impact_label', 'supply_chain_stage_label', 'geographical_region_label', 'temporal_trend_label']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Split the data into features and target variables for each task
features = data['text_description']  # Assuming all tasks use the same text features, adjust as necessary

# Create SBERT embeddings
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
sbert_embeddings = sbert_model.encode(features.tolist())

# Emission Quantity Prediction
emission_quantity = data['emission_quantity'].values  # Replace with actual column

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(sbert_embeddings, emission_quantity)
lin_reg_predictions = lin_reg.predict(sbert_embeddings)
lin_reg_accuracy = lin_reg.score(sbert_embeddings, emission_quantity)
lin_reg_mse = mean_squared_error(emission_quantity, lin_reg_predictions)

# Random Forest Regression
rf_reg = RandomForestRegressor()
rf_reg.fit(sbert_embeddings, emission_quantity)
rf_reg_predictions = rf_reg.predict(sbert_embeddings)
rf_reg_accuracy = rf_reg.score(sbert_embeddings, emission_quantity)
rf_reg_mse = mean_squared_error(emission_quantity, rf_reg_predictions)

# Gradient Boosting Regression
gb_reg = GradientBoostingRegressor()
gb_reg.fit(sbert_embeddings, emission_quantity)
gb_reg_predictions = gb_reg.predict(sbert_embeddings)
gb_reg_accuracy = gb_reg.score(sbert_embeddings, emission_quantity)
gb_reg_mse = mean_squared_error(emission_quantity, gb_reg_predictions)

# Print results for Emission Quantity Prediction
print("Emission Quantity Prediction:")
print(f"Linear Regression - Accuracy: {lin_reg_accuracy:.2f}, MSE: {lin_reg_mse:.4f}")
print(f"Random Forest Regression - Accuracy: {rf_reg_accuracy:.2f}, MSE: {rf_reg_mse:.4f}")
print(f"Gradient Boosting Regression - Accuracy: {gb_reg_accuracy:.2f}, MSE: {gb_reg_mse:.4f}")

# Emission Type Identification
emission_type = data['emission_type_label'].values

# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(sbert_embeddings, emission_type)
log_reg_predictions = log_reg.predict(sbert_embeddings)
log_reg_accuracy = accuracy_score(emission_type, log_reg_predictions)
log_reg_precision = precision_score(emission_type, log_reg_predictions, average='weighted')
log_reg_recall = recall_score(emission_type, log_reg_predictions, average='weighted')
log_reg_f1 = f1_score(emission_type, log_reg_predictions, average='weighted')

# Support Vector Machines
svm = SVC()
svm.fit(sbert_embeddings, emission_type)
svm_predictions = svm.predict(sbert_embeddings)
svm_accuracy = accuracy_score(emission_type, svm_predictions)
svm_precision = precision_score(emission_type, svm_predictions, average='weighted')
svm_recall = recall_score(emission_type, svm_predictions, average='weighted')
svm_f1 = f1_score(emission_type, svm_predictions, average='weighted')

# Decision Trees
tree = DecisionTreeClassifier()
tree.fit(sbert_embeddings, emission_type)
tree_predictions = tree.predict(sbert_embeddings)
tree_accuracy = accuracy_score(emission_type, tree_predictions)
tree_f1 = f1_score(emission_type, tree_predictions, average='weighted')

# Print results for Emission Type Identification
print("\nEmission Type Identification:")
print(f"Logistic Regression - Accuracy: {log_reg_accuracy:.2f}, Precision: {log_reg_precision:.2f}, Recall: {log_reg_recall:.2f}, F1: {log_reg_f1:.2f}")
print(f"Support Vector Machines - Accuracy: {svm_accuracy:.2f}, Precision: {svm_precision:.2f}, Recall: {svm_recall:.2f}, F1: {svm_f1:.2f}")
print(f"Decision Trees - Accuracy: {tree_accuracy:.2f}, F1: {tree_f1:.2f}")

# Impact Assessment
impact = data['impact_label'].values

# Binary Relevance
binary_relevance = MultiOutputClassifier(LogisticRegression())
binary_relevance.fit(sbert_embeddings, impact)
binary_relevance_predictions = binary_relevance.predict(sbert_embeddings)
binary_relevance_accuracy = accuracy_score(impact, binary_relevance_predictions)
binary_relevance_precision = precision_score(impact, binary_relevance_predictions, average='weighted')
binary_relevance_recall = recall_score(impact, binary_relevance_predictions, average='weighted')
binary_relevance_f1 = f1_score(impact, binary_relevance_predictions, average='weighted')

# Label Powerset
from skmultilearn.problem_transform import LabelPowerset
label_powerset = LabelPowerset(LogisticRegression())
label_powerset.fit(sbert_embeddings, impact)
label_powerset_predictions = label_powerset.predict(sbert_embeddings)
label_powerset_accuracy = accuracy_score(impact, label_powerset_predictions)
label_powerset_f1 = f1_score(impact, label_powerset_predictions, average='weighted')

# Print results for Impact Assessment
print("\nImpact Assessment:")
print(f"Binary Relevance - Accuracy: {binary_relevance_accuracy:.2f}, Precision: {binary_relevance_precision:.2f}, Recall: {binary_relevance_recall:.2f}, F1: {binary_relevance_f1:.2f}")
print(f"Label Powerset - Accuracy: {label_powerset_accuracy:.2f}, F1: {label_powerset_f1:.2f}")

# Supply Chain Stage Identification
supply_chain_stage = data['supply_chain_stage_label'].values

# Conditional Random Fields
from sklearn_crfsuite import CRF
crf = CRF()
crf.fit(sbert_embeddings, supply_chain_stage)
crf_predictions = crf.predict(sbert_embeddings)
crf_accuracy = accuracy_score(supply_chain_stage, crf_predictions)
crf_f1 = f1_score(supply_chain_stage, crf_predictions, average='weighted')

# Hidden Markov Models
from hmmlearn import hmm
hmm_model = hmm.GaussianHMM()
hmm_model.fit(sbert_embeddings, supply_chain_stage)
hmm_predictions = hmm_model.predict(sbert_embeddings)
hmm_accuracy = accuracy_score(supply_chain_stage, hmm_predictions)
hmm_f1 = f1_score(supply_chain_stage, hmm_predictions, average='weighted')

# Print results for Supply Chain Stage Identification
print("\nSupply Chain Stage Identification:")
print(f"Conditional Random Fields - Accuracy: {crf_accuracy:.2f}, F1: {crf_f1:.2f}")
print(f"Hidden Markov Models - Accuracy: {hmm_accuracy:.2f}, F1: {hmm_f1:.2f}")

# Geographical Region Classification
geographical_region = data['geographical_region_label'].values

# k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(sbert_embeddings, geographical_region)
knn_predictions = knn.predict(sbert_embeddings)
knn_accuracy = accuracy_score(geographical_region, knn_predictions)
knn_precision = precision_score(geographical_region, knn_predictions, average='weighted')
knn_recall = recall_score(geographical_region, knn_predictions, average='weighted')
knn_f1 = f1_score(geographical_region, knn_predictions, average='weighted')

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(sbert_embeddings, geographical_region)
nb_predictions = nb.predict(sbert_embeddings)
nb_accuracy = accuracy_score(geographical_region, nb_predictions)
nb_precision = precision_score(geographical_region, nb_predictions, average='weighted')
nb_recall = recall_score(geographical_region, nb_predictions, average='weighted')
nb_f1 = f1_score(geographical_region, nb_predictions, average='weighted')

# Print results for Geographical Region Classification
print("\nGeographical Region Classification:")
print(f"k-Nearest Neighbors - Accuracy: {knn_accuracy:.2f}, Precision: {knn_precision:.2f}, Recall: {knn_recall:.2f}, F1: {knn_f1:.2f}")
print(f"Naive Bayes - Accuracy: {nb_accuracy:.2f}, Precision: {nb_precision:.2f}, Recall: {nb_recall:.2f}, F1: {nb_f1:.2f}")

# Temporal Trend Prediction
temporal_trend = data['temporal_trend'].values

# ARIMA
arima_model = ARIMA(temporal_trend, order=(5, 1, 0))
arima_fit = arima_model.fit()
arima_predictions = arima_fit.predict()
arima_mse = mean_squared_error(temporal_trend, arima_predictions)

# Exponential Smoothing
exp_smoothing = ExponentialSmoothing(temporal_trend)
exp_smoothing_fit = exp_smoothing.fit()
exp_smoothing_predictions = exp_smoothing_fit.predict()
exp_smoothing_mse = mean_squared_error(temporal_trend, exp_smoothing_predictions)

# Print results for Temporal Trend Prediction
print("\nTemporal Trend Prediction:")
print(f"ARIMA - MSE: {arima_mse:.4f}")
print(f"Exponential Smoothing - MSE: {exp_smoothing_mse:.4f}")

# TF-IDF and Word2Vec Embeddings
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(features).toarray()

# Word2Vec
word2vec_model = Word2Vec(features.apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)
word2vec_features = np.array([np.mean([word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for text in features])

# SVM Classifier
svm_tfidf = SVC()
svm_tfidf.fit(tfidf_features, emission_type)
svm_tfidf_predictions = svm_tfidf.predict(tfidf_features)
svm_tfidf_accuracy = accuracy_score(emission_type, svm_tfidf_predictions)

svm_word2vec = SVC()
svm_word2vec.fit(word2vec_features, emission_type)
svm_word2vec_predictions = svm_word2vec.predict(word2vec_features)
svm_word2vec_accuracy = accuracy_score(emission_type, svm_word2vec_predictions)

# Random Forest Classifier
rf_tfidf = RandomForestClassifier()
rf_tfidf.fit(tfidf_features, emission_type)
rf_tfidf_predictions = rf_tfidf.predict(tfidf_features)
rf_tfidf_accuracy = accuracy_score(emission_type, rf_tfidf_predictions)

rf_word2vec = RandomForestClassifier()
rf_word2vec.fit(word2vec_features, emission_type)
rf_word2vec_predictions = rf_word2vec.predict(word2vec_features)
rf_word2vec_accuracy = accuracy_score(emission_type, rf_word2vec_predictions)

# Print results for TF-IDF and Word2Vec
print("\nTF-IDF and Word2Vec Embeddings:")
print(f"SVM (TF-IDF) - Accuracy: {svm_tfidf_accuracy:.2f}")
print(f"SVM (Word2Vec) - Accuracy: {svm_word2vec_accuracy:.2f}")
print(f"Random Forest (TF-IDF) - Accuracy: {rf_tfidf_accuracy:.2f}")
print(f"Random Forest (Word2Vec) - Accuracy: {rf_word2vec_accuracy:.2f}")

# Plotting results (example for Emission Type Identification)
tasks = ['Emission Quantity Prediction', 'Emission Type Identification', 'Impact Assessment', 'Supply Chain Stage Identification', 'Geographical Region Classification', 'Temporal Trend Prediction']
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

results = {
    'Emission Type Identification': [log_reg_accuracy, log_reg_precision, log_reg_recall, log_reg_f1],
    'Supply Chain Stage Identification': [crf_accuracy, None, None, crf_f1],
    'Geographical Region Classification': [knn_accuracy, knn_precision, knn_recall, knn_f1]
    # Add other results similarly
}

for metric in metrics:
    for task, values in results.items():
        plt.plot(tasks, values, marker='o', label=task)

plt.xlabel('Tasks')
plt.ylabel('Scores')
plt.title('Comparison of Classification Tasks')
plt.legend()
plt.xticks(rotation=45)
plt.show()


Results of fine-tuning BERT for SCOPE 3 emission estimation